In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
import gdown
import pandas as pd

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [5]:
url = "https://docs.google.com/spreadsheets/d/1dSgqL4nAQGWwyCzgndcdYP_0dRk8uEEioq-trKsWt6U/export?format=xlsx"

output_file = "latoken_dataset.xlsx"

gdown.download(url, output_file, quiet=False)

df = pd.read_excel(output_file, sheet_name="Hoja 1")

names = ["question", "answer"]

df.columns = names

df

/usr/local/lib/python3.11/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/1dSgqL4nAQGWwyCzgndcdYP_0dRk8uEEioq-trKsWt6U/export?format=xlsx
To: /content/latoken_dataset.xlsx
26.1kB [00:00, 1.34MB/s]


,question,answer
0,Какие цели у Латокен?,Цели Латокен - это научиться листить лучшие то...
1,Какие преимущества у Латокен?,Мы помогаем людям стать со-владельцами стартап...
2,Какая статистика у Латокен?,Мы являемся номер 1 по числу активов для трейд...
3,Преимущество работы в Латокен?,Наши преимущества - это рост на глобальном рын...
4,Каким должен быть сотрудник Латокен?,Работник должен отвечать за пользу клиентов. О...
...,...,...
126,Почему именно хакатоны?,Потому что даже гонщику Формулы-1 нужен пит-ст...
127,Что делать на хакатоне?,"Создавать будущее, знакомиться с командой визи..."
128,Почему Латокен?,Потому что мы не просто биржа с 3000+ активов....
129,Какая атмосфера в команде?,"Представь Кремниевую долину, Хогвартс и космич..."


In [6]:
formatted_data = []

for index, row in df.iterrows():
    conversation = [
        {"from": "human", "value": row["question"]},
        {"from": "gpt", "value": row["answer"]}
    ]
    formatted_data.append(conversation)

# Check the formatted data
print(formatted_data)

[[{'from': 'human', 'value': 'Какие цели у Латокен?'}, {'from': 'gpt', 'value': 'Цели Латокен - это научиться листить лучшие токены первыми, а также сделать их ликвидными и легко понятными для торговли. Для этого мы используем ИИ, автоматизируя и расширяя продажи, а также обогащая информацию о токенах данными ончейн и социальными данными. А самое главное, мы строим команду, которая на это способна. Мы считаем, что программировать научится ИИ, поэтому характер важнее навыков разработки. Мы помогаем людям стать со-владельцами стартапов, которые могут стать основой будущей экономики.'}], [{'from': 'human', 'value': 'Какие преимущества у Латокен?'}, {'from': 'gpt', 'value': 'Мы помогаем людям стать со-владельцами стартапов, которые могут стать основой будущей экономики. Этим со-владельцам сиюминутно выгодно кооперироваться для создания лучшего будущего вместо войн.'}], [{'from': 'human', 'value': 'Какая статистика у Латокен?'}, {'from': 'gpt', 'value': 'Мы являемся номер 1 по числу активов

In [7]:
formatted_data[1]

[{'from': 'human', 'value': 'Какие преимущества у Латокен?'},
 {'from': 'gpt',
  'value': 'Мы помогаем людям стать со-владельцами стартапов, которые могут стать основой будущей экономики. Этим со-владельцам сиюминутно выгодно кооперироваться для создания лучшего будущего вместо войн.'}]

In [8]:
from unsloth.chat_templates import get_chat_template

# Assuming `tokenizer` is already defined
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

# Convert the formatted data into a Hugging Face dataset
from datasets import Dataset

dataset = Dataset.from_dict({"conversations": formatted_data})
dataset = dataset.map(apply_template, batched=True)

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


Map:   0%|          | 0/131 [00:00<?, ? examples/s]

In [9]:
dataset[1]

{'conversations': [{'from': 'human', 'value': 'Какие преимущества у Латокен?'},
  {'from': 'gpt',
   'value': 'Мы помогаем людям стать со-владельцами стартапов, которые могут стать основой будущей экономики. Этим со-владельцам сиюминутно выгодно кооперироваться для создания лучшего будущего вместо войн.'}],
 'text': '<|im_start|>user\nКакие преимущества у Латокен?<|im_end|>\n<|im_start|>assistant\nМы помогаем людям стать со-владельцами стартапов, которые могут стать основой будущей экономики. Этим со-владельцам сиюминутно выгодно кооперироваться для создания лучшего будущего вместо войн.<|im_end|>\n'}

In [10]:
train_dataset, val_dataset = dataset.train_test_split(test_size=0.2).values()

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 8,
        num_train_epochs = 8,
        #max_steps = None,
        learning_rate = 6e-4,
        max_grad_norm=1.0,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc,
        #load_best_model_at_end=True,
        #metric_for_best_model="eval_loss",
        #greater_is_better=False,
    ),
)

Map (num_proc=2):   0%|          | 0/104 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/27 [00:00<?, ? examples/s]

In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.996 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 104 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 104
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,3.525200
2,3.740200
3,2.398200
4,2.130200
5,1.723500
6,1.778600
7,1.646900
8,1.479200
9,1.497000
10,1.421100


In [14]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

686.1387 seconds used for training.
11.44 minutes used for training.
Peak reserved memory = 7.076 GB.
Peak reserved memory for training = 1.08 GB.
Peak reserved memory % of max memory = 47.979 %.
Peak reserved memory for training % of max memory = 7.323 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [53]:
from transformers import TextStreamer

model = FastLanguageModel.for_inference(model)

messages = [
    {"from": "human", "value": "Что ты знаешь про суперпозицию?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True)

<|im_start|>user
Что ты знаешь про суперпозицию?<|im_end|>
<|im_start|>assistant
Это когда два токена на одной неделе показывают одинаковые результаты. Возможно, это связано с тем, как мы обрабатываем данные.<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [16]:
from getpass import getpass
hf_token = = getpass("Enter hf token: ")

In [ ]:
model.save_pretrained("content/drive/MyDrive/lora_model") # Local saving
tokenizer.save_pretrained("content/drive/MyDrive/lora_tokenizer")


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('content/drive/MyDrive/lora_tokenizer/tokenizer_config.json',
 'content/drive/MyDrive/lora_tokenizer/special_tokens_map.json',
 'content/drive/MyDrive/lora_tokenizer/tokenizer.json')

In [54]:
model.push_to_hub("t8ix/lora_latoken_llama_new", token = hf_token) # Online saving
tokenizer.push_to_hub("t8ix/lora_latoken_llama_new", token = hf_token) # Online saving

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/t8ix/lora_latoken_llama_new


No files have been modified since last commit. Skipping to prevent empty commit.
